In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/embedding-for-task-assigning-module-3/embeddings/4_NN_FLINK_10_assignees_features/y_test.csv
/kaggle/input/embedding-for-task-assigning-module-3/embeddings/4_NN_FLINK_10_assignees_features/X_test.csv
/kaggle/input/embedding-for-task-assigning-module-3/embeddings/4_NN_FLINK_10_assignees_features/y_train.csv
/kaggle/input/embedding-for-task-assigning-module-3/embeddings/4_NN_FLINK_10_assignees_features/X_train.csv
/kaggle/input/embedding-for-task-assigning-module-3/embeddings/4_NN_FLINK_10_assignees_features/embedding_matrix.npy
/kaggle/input/embedding-for-task-assigning-module-3/embeddings/4_NN_FLINK_10_assignees_features/word_index.json
/kaggle/input/embedding-for-task-assigning-module-3/embeddings/4_NN_AMBARI_5_assignees_features/y_test.csv
/kaggle/input/embedding-for-task-assigning-module-3/embeddings/4_NN_AMBARI_5_assignees_features/X_test.csv
/kaggle/input/embedding-for-task-assigning-module-3/embeddings/4_NN_AMBARI_5_assignees_features/y_train.csv
/kaggle/input/embed

In [2]:
import os
import numpy as np
import pandas as pd
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


2025-06-09 18:11:31.817906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749492692.011693      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749492692.066050      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
EMBEDDING_DIM = 100
MAX_NUM_WORDS = 20000
trainable_data_folder = "/kaggle/input/embedding-for-task-assigning-module-3/embeddings"
output_base_folder = "/kaggle/working/nn_results/"
os.makedirs(output_base_folder, exist_ok=True)

In [7]:
feature_dirs = [d for d in os.listdir(trainable_data_folder) if d.startswith("4_NN_")]
    
for feature_dir in feature_dirs:
    print(f"\\n=================================================")
    print(f"Processing dataset: {feature_dir}")
    print(f"=================================================")
    
    try:
        full_path = os.path.join(trainable_data_folder, feature_dir)
        full_output_path = os.path.join(output_base_folder, feature_dir)
        os.makedirs(full_output_path, exist_ok=True)
        # --- 1. Load All Pre-processed Data ---
        print("  Loading pre-processed data and embedding matrix...")
        X_train = pd.read_csv(os.path.join(full_path, 'X_train.csv'), header=None).values
        X_test = pd.read_csv(os.path.join(full_path, 'X_test.csv'), header=None).values
        y_train_df = pd.read_csv(os.path.join(full_path, 'y_train.csv'))
        y_test_df = pd.read_csv(os.path.join(full_path, 'y_test.csv'))
        embedding_matrix = np.load(os.path.join(full_path, 'embedding_matrix.npy'))
        
        with open(os.path.join(full_path, 'word_index.json'), 'r') as f:
            word_index = json.load(f)

        # One-hot encode labels
        y_train = pd.get_dummies(y_train_df['assignee_id']).values
        y_test = pd.get_dummies(y_test_df['assignee_id']).values
        num_classes = y_train.shape[1]
        num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
        print(f"  Data loaded. Number of classes (assignees): {num_classes}")
        
        # --- 2. Define the Neural Network Model ---
        print("  Defining the LSTM model architecture...")
        model = Sequential([
                # FIX 2: Removed the deprecated 'input_length' argument
                Embedding(num_words,
                          EMBEDDING_DIM,
                          weights=[embedding_matrix],
                          trainable=False),
                LSTM(128, dropout=0.2, recurrent_dropout=0.2),
                Dense(64, activation='relu'),
                Dropout(0.5),
                Dense(num_classes, activation='softmax')
            ])

        # --- 3. Compile the Model ---
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.summary()

        # --- 4. Train the Model ---
        print("\\n  Training the model...")
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
        
        history = model.fit(X_train, y_train,
                            epochs=20,
                            batch_size=64,
                            validation_data=(X_test, y_test),
                            callbacks=[early_stopping],
                            verbose=1)

        # --- 5. Evaluate and Save Results ---
        print("\\n  Evaluating final model on test data...")
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        y_pred_proba = model.predict(X_test)
        y_pred = np.argmax(y_pred_proba, axis=1)
        y_test_labels = np.argmax(y_test, axis=1)

        results = {
            "loss": loss,
            "accuracy": accuracy,
            "precision_macro": precision_score(y_test_labels, y_pred, average='macro', zero_division=0),
            "recall_macro": recall_score(y_test_labels, y_pred, average='macro', zero_division=0),
            "f1_score_macro": f1_score(y_test_labels, y_pred, average='macro', zero_division=0)
        }
        print(f"  Test Accuracy: {results['accuracy']:.4f}")
        
        results_path = os.path.join(full_output_path, 'results_nn.json')
        with open(results_path, 'w') as f:
            json.dump(results, f, indent=4)
        
        model.save(os.path.join(full_output_path, 'model.h5'))
        print(f"  Saved performance metrics to {results_path}")
        print(f"  Saved trained model to {os.path.join(full_output_path, 'model.h5')}")

    except Exception as e:
        print(f"  [ERROR] An unexpected error occurred while processing {feature_dir}: {e}")

print("\\n--- All NN model training complete! ---")

\n=================================================
Processing dataset: 4_NN_FLINK_10_assignees_features
  Loading pre-processed data and embedding matrix...
  Data loaded. Number of classes (assignees): 10
  Defining the LSTM model architecture...


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ ?                           │         318,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 318,000 (1.21 MB)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_24 (Embedding)             │ ?                           │         262,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_24 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_48 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_49 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 262,500 (1.00 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 262,500 (1.00 MB)

\n  Training the model...
Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 620ms/step - accuracy: 0.2412 - loss: 1.6161 - val_accuracy: 0.1625 - val_loss: 1.6062
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 394ms/step - accuracy: 0.2722 - loss: 1.5834 - val_accuracy: 0.2125 - val_loss: 1.5908
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 395ms/step - accuracy: 0.3035 - loss: 1.5650 - val_accuracy: 0.1750 - val_loss: 1.5974
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 397ms/step - accuracy: 0.3432 - loss: 1.5245 - val_accuracy: 0.1625 - val_loss: 1.5935
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 393ms/step - accuracy: 0.4080 - loss: 1.4877 - val_accuracy: 0.2625 - val_loss: 1.5699
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 398ms/step - accuracy: 0.4232 - loss: 1.4541 - val_accuracy: 0.3500 - val_loss: 1.5438
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 395ms/step - accuracy: 0.4619 - loss: 1.4076 - val_accuracy: 0.3125 - val_loss: 1.5406
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 397ms/step - accuracy: 0.4496 - loss: 1.3342 - val_ac

In [8]:
!zip -r nn_results.zip /kaggle/working/nn_results

  adding: kaggle/working/nn_results/ (stored 0%)
  adding: kaggle/working/nn_results/4_NN_FLINK_5_assignees_features/ (stored 0%)
  adding: kaggle/working/nn_results/4_NN_FLINK_5_assignees_features/model.h5 (deflated 16%)
  adding: kaggle/working/nn_results/4_NN_FLINK_5_assignees_features/results_nn.json (deflated 34%)
  adding: kaggle/working/nn_results/4_NN_IMPALA_5_assignees_features/ (stored 0%)
  adding: kaggle/working/nn_results/4_NN_IMPALA_5_assignees_features/model.h5 (deflated 17%)
  adding: kaggle/working/nn_results/4_NN_IMPALA_5_assignees_features/results_nn.json (deflated 34%)
  adding: kaggle/working/nn_results/4_NN_GEODE_5_assignees_features/ (stored 0%)
  adding: kaggle/working/nn_results/4_NN_GEODE_5_assignees_features/model.h5 (deflated 20%)
  adding: kaggle/working/nn_results/4_NN_GEODE_5_assignees_features/results_nn.json (deflated 34%)
  adding: kaggle/working/nn_results/4_NN_ARROW_5_assignees_features/ (stored 0%)
  adding: kaggle/working/nn_results/4_NN_ARROW_5_as